# <img style="float: left; padding: 10px 0px 0px 0px;" src="https://www.crcom.gov.co/sites/default/files/webcrc/noticias/images/Logo-CRC.jpg"  width="300" /> Prueba técnica CRC
**Ivan Mendoza**<br>

---

Comenzamos importando los paquetes necesarios:

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

### Objetivo:

El objetivo de esta prueba es conocer tus destrezas en el análisis de información, como parte del proceso en el cual estás participando.

En la base de datos denominada `Base_Sitios` encontrará información asociada a la ubicación de las estaciones base con que cuentan los operadores móviles. Esta base de datos contiene la información asociada al municipio, estación base, operador dueño de la estación base y cantidad de sectores por cada estación; tenga en cuenta que encontrará una variable denominada *DESC_TECNOLOGIA*, la cual muestra la información del tipo de tecnología soportada por la estación base, considere que los servicios de *voz* y *SMS* pueden cursar sobre las tecnologías $GSM$, $UMTS$ y $HSPA$ y el servicio de datos puede cursarse por la tecnología *LTE*.

En la base de datos denominada `Tráfico_mensual`, encontrará la información asociada al tráfico cursado para cada uno de los servicios (*voz*, *SMS* y *Datos*) por cada operador de origen, entendido como aquel que cursa tráfico en la red del operador visitado. Para aclarar, hay operadores que cuentan con red, definida para este caso particular como estaciones base, la cual pueden alquilar a otros operadores para que cursen su tráfico.

Teniendo en cuenta lo anterior, le solicitamos: i) Unir las bases de datos previamente descritas; ii) en no más de dos páginas y apoyándose en algunas gráficas, presente las conclusiones, a su juicio las más relevantes, que puede abstraer del conjunto de datos.

Te solicitamos enviar tus resultados a más tardar el viernes 24 de marzo a las 5:00pm incluyendo el archivo en el cual presentas las conclusiones, la base de datos unida, el código que empleaste para desarrollar la prueba y el programa en el cual lo desarrollaste.

## 1. Cargamos las bases

### 1.1 Vemos la descripción de la base `Tráfico_mensual`

Cargamos la hoja *Diccionario* donde podremos la descripción de la base `Tráfico_mensual` en la hoja *Base_tráfico*.

Leamos el archivo con `Pandas`.

In [26]:
#Ajustamos el parametro 'display.max_colwidth' para poder leer toda la información de la columna Descripción
pd.set_option('display.max_colwidth', 200)

# Cargamos la primer hoja con la descripción de las variables y visualizamos el contenido
df_tra_des = pd.read_excel("Tráfico_mensual.xlsx", 
                           sheet_name= "Diccionario")
df_tra_des

,Variable,Descripción
0,AÑO,el año de estudio para esta información es 2018
1,MES,mes de estudio
2,TRIMESTRE,trimestre de estudio
3,ID_MUNICIPIO,corresponde al código DIVIPOLA. El valor NA es información en la que el PRV no indico el código del municipio
4,NOM_MUNICIPIO,corresponde al nombre del municipio
5,PRO,es el Proveedor de Red Origen - Es el operador dueño del usuario que pide prestada la red
6,PRV,es el Proveedor de Red Visitada - Es el operador dueño de la red
7,SERVICIO,"corresponde a los servicios de Voz, Datos y SMS que reportan los PRV en la medida RAN"
8,TRÁFICO,"corresponde al valor de tráfico reportado según servicio. Datos=Megabytes, SMS=cantidad de mensajes, Voz= minutos"


### 1.1.1 Cargamos la base `Tráfico_mensual`

Cargamos la hoja *Base_tráfico* donde podremos ver la información de la base `Tráfico_mensual`.

Leamos el archivo con `Pandas`.

In [46]:
#Ajustamos el parametro 'display.max_colwidth' para poder leer toda la información
pd.set_option('display.max_colwidth', 50)

# Validamos que la información cargada tenga el formato correcto 
#(ajustamos el formato de la columna ID_MUNICIPIO para leerlo como string y definimos el decimal como como ",")
#Además, revisamos que no se identifiquen valores nulos.

df_tra = pd.read_excel("Tráfico_mensual.xlsx", 
                       sheet_name= "Base_tráfico", 
                       dtype ={"ID_MUNICIPIO":str}, 
                       decimal= ",")
df_tra.head()

,AÑO,MES,TRIMESTRE,ID_MUNICIPIO,NOM_MUNICIPIO,PRV,PRO,SERVICIO,TRÁFICO
0,2018,10,4,11001,"BOGOTÁ, D.C.",Operador 6,Operador 2,VOZ,1.088575e+08
1,2018,11,4,11001,"BOGOTÁ, D.C.",Operador 6,Operador 2,VOZ,1.001573e+08
2,2018,9,3,11001,"BOGOTÁ, D.C.",Operador 6,Operador 2,VOZ,9.908555e+07
3,2018,8,3,11001,"BOGOTÁ, D.C.",Operador 6,Operador 2,VOZ,9.832873e+07
4,2018,5,2,11001,"BOGOTÁ, D.C.",Operador 6,Operador 2,VOZ,9.766652e+07


In [47]:
df_tra.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 153829 entries, 0 to 153828
Data columns (total 9 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   AÑO            153829 non-null  int64  
 1   MES            153829 non-null  int64  
 2   TRIMESTRE      153829 non-null  int64  
 3   ID_MUNICIPIO   153829 non-null  object 
 4   NOM_MUNICIPIO  153829 non-null  object 
 5   PRV            153829 non-null  object 
 6   PRO            153829 non-null  object 
 7   SERVICIO       153829 non-null  object 
 8   TRÁFICO        153829 non-null  float64
dtypes: float64(1), int64(3), object(5)
memory usage: 10.6+ MB


No se identifican valores nulos a tratar, por tanto seguimos con el cargue de la base `Base_Sitios`.

In [85]:
df_tra[["SERVICIO","NOM_MUNICIPIO"]].groupby("SERVICIO").count()

,NOM_MUNICIPIO
SERVICIO,
DATOS,46497
SMS,48452
VOZ,58880


In [149]:
df_tra[["NOM_MUNICIPIO","TRÁFICO"]].groupby("NOM_MUNICIPIO").count()

,TRÁFICO
NOM_MUNICIPIO,
ABEJORRAL,172
ABRIAQUÍ,102
ACACÍAS,194
ACANDÍ,157
ACEVEDO,162
...,...
ZIPAQUIRÁ,165
ZONA BANANERA,182
ÁBREGO,163


In [98]:
#Revisamos los operadores dueños de la red, y vemos que solo 3 operadores son dueños de la red
df_tra[["PRV","NOM_MUNICIPIO"]].groupby("PRV").count()

,NOM_MUNICIPIO
PRV,
Operador 1,78538
Operador 4,42228
Operador 6,33063


In [285]:
#Revisamos los operadores dueños de los usuarios, y vemos que hay 4 operadores
df_tra[["PRV","NOM_MUNICIPIO"]].groupby("PRV").count()

,NOM_MUNICIPIO
PRV,
Operador 1,78538
Operador 4,42228
Operador 6,33063


In [256]:
# Creamos una columnas para cruzar con la información de la base de sitios
def servicio_C(x):
    """ columna necesaria para cruzar con la información de bases"""
    if x == "DATOS":
        return "DATOS"
    elif (x == "SMS") or (x == "VOZ"):
        return "SMS/VOZ"
    else:
        return "No registra información"

In [257]:
# Aplicamos la función creada para crear la nueva columna
df_tra["SERVICIO_C"] = df_tra["SERVICIO"].apply(servicio_C)

In [258]:
df_tra.head()

,AÑO,MES,TRIMESTRE,ID_MUNICIPIO,NOM_MUNICIPIO,PRV,PRO,SERVICIO,TRÁFICO,SERVICIO_C
0,2018,10,4,11001,"BOGOTÁ, D.C.",Operador 6,Operador 2,VOZ,1.088575e+08,SMS/VOZ
1,2018,11,4,11001,"BOGOTÁ, D.C.",Operador 6,Operador 2,VOZ,1.001573e+08,SMS/VOZ
2,2018,9,3,11001,"BOGOTÁ, D.C.",Operador 6,Operador 2,VOZ,9.908555e+07,SMS/VOZ
3,2018,8,3,11001,"BOGOTÁ, D.C.",Operador 6,Operador 2,VOZ,9.832873e+07,SMS/VOZ
4,2018,5,2,11001,"BOGOTÁ, D.C.",Operador 6,Operador 2,VOZ,9.766652e+07,SMS/VOZ


In [259]:
df_tra[["SERVICIO_C","NOM_MUNICIPIO"]].groupby("SERVICIO_C").count()

,NOM_MUNICIPIO
SERVICIO_C,
DATOS,46497
SMS/VOZ,107332


### 1.2. Cargamos la base `Base_Sitios`

Cargamos la hoja *Base_tráfico* donde podremos ver la información de la base `Base_Sitios`.

Leamos el archivo con `Pandas`.

In [204]:
# Validamos que la información cargada tenga el formato correcto 
#(ajustamos el encoding ya que los datos tienen tildes y caracteres del latin; 
#también, definimos el separador como punto y como";" y le asignamos el tipo string a las columnas ID);
#adicionalmente, revisamos que no se identifiquen valores nulos.

df_sit = pd.read_csv("Base_Sitios.csv", 
                     encoding= "latin-1", 
                     sep = ";", 
                     dtype = {"ID_EMPRESA":str, 
                              "CODIGO_SITIO":str, 
                              "ID_MUNICIPIO":str, 
                              "ID_DEPTO":str, 
                              "ID_TECNOLOGIA_INSTALADA":str, 
                              "AZIMUT":str, 
                              "SECTOR":str})
df_sit.head()

,ANNO,TRIMESTRE,ID_EMPRESA,PROVEEDOR,CODIGO_SITIO,ID_MUNICIPIO,ID_DEPTO,DEPARTAMENTO,MUNICIPIO,ID_TECNOLOGIA_INSTALADA,DESC_TECNOLOGIA_INSTALADA,SECTOR,AZIMUT
0,2017.0,4.0,111,Operador 1,1259,91540,91,AMAZONAS,PUERTO NARIÑO,2,GSM,3,110
1,2017.0,4.0,111,Operador 1,1259,91540,91,AMAZONAS,PUERTO NARIÑO,2,GSM,2,160
2,2017.0,4.0,111,Operador 1,1259,91540,91,AMAZONAS,PUERTO NARIÑO,2,GSM,1,330
3,2017.0,4.0,111,Operador 1,1260,91263,91,AMAZONAS,EL ENCANTO,4,HSPA,3,90
4,2017.0,4.0,111,Operador 1,1260,91263,91,AMAZONAS,EL ENCANTO,4,HSPA,2,210


In [205]:
#Vemos que existen valores nulos en toda la tabla, así que procedemos a validar si son filas completas
df_sit.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 488765 entries, 0 to 488764
Data columns (total 13 columns):
 #   Column                     Non-Null Count   Dtype  
---  ------                     --------------   -----  
 0   ANNO                       487921 non-null  float64
 1   TRIMESTRE                  487921 non-null  float64
 2   ID_EMPRESA                 487921 non-null  object 
 3   PROVEEDOR                  487921 non-null  object 
 4   CODIGO_SITIO               487921 non-null  object 
 5   ID_MUNICIPIO               487921 non-null  object 
 6   ID_DEPTO                   487921 non-null  object 
 7   DEPARTAMENTO               487921 non-null  object 
 8   MUNICIPIO                  487921 non-null  object 
 9   ID_TECNOLOGIA_INSTALADA    487921 non-null  object 
 10  DESC_TECNOLOGIA_INSTALADA  487921 non-null  object 
 11  SECTOR                     487921 non-null  object 
 12  AZIMUT                     487921 non-null  object 
dtypes: float64(2), object(11)
mem

In [206]:
# Usar isnull() y any() para identificar filas vacías
filas_vacias = df_sit.isnull().any(axis=1)

# Filtramos el DataFrame original por las filas vacías
df_vacio = df_sit[filas_vacias]

# Vemos que hay 844 filas totalmente vacias, así que procedemos a eliminarlas
df_vacio

,ANNO,TRIMESTRE,ID_EMPRESA,PROVEEDOR,CODIGO_SITIO,ID_MUNICIPIO,ID_DEPTO,DEPARTAMENTO,MUNICIPIO,ID_TECNOLOGIA_INSTALADA,DESC_TECNOLOGIA_INSTALADA,SECTOR,AZIMUT
109674,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
109676,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
109677,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
109679,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
109680,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
378930,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
378931,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
378932,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
378934,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [207]:
# procedemos a eliminar las filas totalmente vacias
df_sit = df_sit.dropna(how="all", axis=0)
df_sit.head()

,ANNO,TRIMESTRE,ID_EMPRESA,PROVEEDOR,CODIGO_SITIO,ID_MUNICIPIO,ID_DEPTO,DEPARTAMENTO,MUNICIPIO,ID_TECNOLOGIA_INSTALADA,DESC_TECNOLOGIA_INSTALADA,SECTOR,AZIMUT
0,2017.0,4.0,111,Operador 1,1259,91540,91,AMAZONAS,PUERTO NARIÑO,2,GSM,3,110
1,2017.0,4.0,111,Operador 1,1259,91540,91,AMAZONAS,PUERTO NARIÑO,2,GSM,2,160
2,2017.0,4.0,111,Operador 1,1259,91540,91,AMAZONAS,PUERTO NARIÑO,2,GSM,1,330
3,2017.0,4.0,111,Operador 1,1260,91263,91,AMAZONAS,EL ENCANTO,4,HSPA,3,90
4,2017.0,4.0,111,Operador 1,1260,91263,91,AMAZONAS,EL ENCANTO,4,HSPA,2,210


In [208]:
# Vemos que ya no existen filas vacias y que el DataFrame esta limpio de nulos
df_sit.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 487921 entries, 0 to 488764
Data columns (total 13 columns):
 #   Column                     Non-Null Count   Dtype  
---  ------                     --------------   -----  
 0   ANNO                       487921 non-null  float64
 1   TRIMESTRE                  487921 non-null  float64
 2   ID_EMPRESA                 487921 non-null  object 
 3   PROVEEDOR                  487921 non-null  object 
 4   CODIGO_SITIO               487921 non-null  object 
 5   ID_MUNICIPIO               487921 non-null  object 
 6   ID_DEPTO                   487921 non-null  object 
 7   DEPARTAMENTO               487921 non-null  object 
 8   MUNICIPIO                  487921 non-null  object 
 9   ID_TECNOLOGIA_INSTALADA    487921 non-null  object 
 10  DESC_TECNOLOGIA_INSTALADA  487921 non-null  object 
 11  SECTOR                     487921 non-null  object 
 12  AZIMUT                     487921 non-null  object 
dtypes: float64(2), object(11)
mem

In [283]:
df_sit[["PROVEEDOR","CODIGO_SITIO"]].groupby("PROVEEDOR").count()

,CODIGO_SITIO
PROVEEDOR,
Operador 1,232761
Operador 2,4924
Operador 3,97910
Operador 4,94588
Operador 5,57374
Operador 6,364


,CODIGO_SITIO
PROVEEDOR,
Operador 1,47401
Operador 2,2089
Operador 3,18255
Operador 4,23386
Operador 5,11559
Operador 6,168


In [209]:
# Vemos los tipos de tecnologias instaldas
df_sit[["DESC_TECNOLOGIA_INSTALADA","CODIGO_SITIO"]].groupby("DESC_TECNOLOGIA_INSTALADA").count()

,CODIGO_SITIO
DESC_TECNOLOGIA_INSTALADA,
GSM,116356
HSPA,145463
LTE,84114
UMTS,141988


In [210]:
# Creamos una columnas para estandarizar la información de las tecnologias con el servicio prestado
def servicio_tecnologia(x):
    """ considere que los servicios de voz y SMS pueden cursar sobre las tecnologías 
    GSM, UMTS y HSPA y el servicio de datos puede cursarse por la tecnología LTE"""
    if x == "LTE":
        return "DATOS"
    elif (x == "GSM") or (x == "UMTS") or (x == "HSPA"):
        return "SMS/VOZ"
    else:
        return "No registra información"

In [211]:
# Aplicamos la función creada para crear la nueva columna
df_sit["SERVICIO"] = df_sit["DESC_TECNOLOGIA_INSTALADA"].apply(servicio_tecnologia)

In [212]:
# Vemos los tipos de servicios y vemos que todas las columnas tenían informaición
df_sit[["SERVICIO","CODIGO_SITIO"]].groupby("SERVICIO").count()

,CODIGO_SITIO
SERVICIO,
DATOS,84114
SMS/VOZ,403807


In [213]:
df_sit.columns

Index(['ANNO', 'TRIMESTRE', 'ID_EMPRESA', 'PROVEEDOR', 'CODIGO_SITIO', 'ID_MUNICIPIO', 'ID_DEPTO', 'DEPARTAMENTO', 'MUNICIPIO', 'ID_TECNOLOGIA_INSTALADA', 'DESC_TECNOLOGIA_INSTALADA', 'SECTOR', 'AZIMUT', 'SERVICIO'], dtype='object')

In [288]:
# Agrupamos las tecnologías, ids, sectores, azimut y servicios (Para mantener la información)
#creadas a nivel de año, trimestre, proveedor, departamento, municipio
df_sit_agru = df_sit.groupby([
    "PROVEEDOR", 
    "ID_MUNICIPIO", 
    "ANNO", 
    "ID_EMPRESA", 
    "ID_DEPTO", 
    "TRIMESTRE", 
    "MUNICIPIO", 
    "DEPARTAMENTO", 
    "SERVICIO"]).apply(
    lambda x: pd.Series({
        "DESC_TECNOLOGIA_INSTALADA": ", ".join(set(x["DESC_TECNOLOGIA_INSTALADA"].astype(str).dropna())),
        "AZIMUT": ", ".join(set(x["AZIMUT"].astype(str).dropna())),
        "ID_TECNOLOGIA_INSTALADA": ", ".join(set(x["ID_TECNOLOGIA_INSTALADA"].astype(str).dropna())),
        "SECTOR": ", ".join(set(x["SECTOR"].astype(str).dropna())),
        "CODIGO_SITIO": ", ".join(set(x["CODIGO_SITIO"].astype(str).dropna())),
    })).reset_index()

In [289]:
df_sit_agru

,PROVEEDOR,ID_MUNICIPIO,ANNO,ID_EMPRESA,ID_DEPTO,TRIMESTRE,MUNICIPIO,DEPARTAMENTO,SERVICIO,DESC_TECNOLOGIA_INSTALADA,AZIMUT,ID_TECNOLOGIA_INSTALADA,SECTOR,CODIGO_SITIO
0,Operador 1,11001,2017.0,111,11,4.0,"BOGOTÁ, D.C.",BOGOTÁ D.C.,DATOS,LTE,"40, 55, 145, 315, 205, 155, 230, 4, 335, 170, ...",5,"5, 7, 4, 3, 2, 6, 8, 1, 10, 9","6102, 4916, 5618, 5118, 4728, 4587, 4997, 5128..."
1,Operador 1,11001,2017.0,111,11,4.0,"BOGOTÁ, D.C.",BOGOTÁ D.C.,SMS/VOZ,"HSPA, GSM","302, 40, 55, 145, 315, 39, 205, 155, 230, 76, ...","4, 2","16, 22, 35, 17, 11, 40, 19, 29, 2, 21, 25, 30,...","5179, 5561, 4611, 4728, 5128, 5579, 5061, 4844..."
2,Operador 1,11001,2018.0,111,11,4.0,"BOGOTÁ, D.C.",BOGOTÁ D.C.,DATOS,LTE,"40, 55, 145, 315, 155, 205, 230, 4, 335, 90, 2...",5,"16, 67, 22, 63, 35, 68, 17, 11, 41, 64, 40, 29...","6102, 4916, 5618, 5118, 4611, 11287, 4728, 458..."
3,Operador 1,11001,2018.0,111,11,4.0,"BOGOTÁ, D.C.",BOGOTÁ D.C.,SMS/VOZ,"HSPA, GSM","40, 55, 145, 315, 39, 155, 205, 230, 76, 335, ...","4, 2","16, 67, 63, 64, 40, 2, 55, 38, 39, 48, 53, 57,...","5179, 5561, 4611, 4728, 5128, 5579, 5061, 4844..."
4,Operador 1,13001,2017.0,111,13,4.0,CARTAGENA,BOLÍVAR,DATOS,LTE,"270, 35, 250, 40, 180, 150, 220, 145, 115, 30,...",5,"3, 2, 1","9834, 9838, 9755, 9853, 9861, 9781, 9709, 9736..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9029,Operador 6,66170,2017.0,116,66,4.0,DOSQUEBRADAS,RISARALDA,DATOS,LTE,"120, 30, 250",5,"3, 2, 1",2227
9030,Operador 6,68001,2017.0,116,68,4.0,BUCARAMANGA,SANTANDER,DATOS,LTE,"200, 205, 75, 180, 330, 310, 100, 45",5,"3, 2, 1","2117, 2119, 2118"
9031,Operador 6,76001,2017.0,116,76,4.0,CALI,VALLE DEL CAUCA,DATOS,LTE,"270, 35, 40, 360, 220, 145, 30, 335, 80, 100, ...",5,"4, 3, 2, 1","2178, 2193, 2195, 2184, 2197, 2180, 2190, 2185..."
9032,Operador 6,76520,2017.0,116,76,4.0,PALMIRA,VALLE DEL CAUCA,DATOS,LTE,"120, 20, 240",5,"3, 2, 1",2229


In [290]:
# Vemos que existen codigos DANE que se encuentran en el DataFrame de trafico mensual, pero no el de estaciones base, 
#por lo cual antes de realizar el cruce, vamos guardar esta información
mask_fil = df_tra["ID_MUNICIPIO"].isin(df_sit_agru["ID_MUNICIPIO"])

# Filtrar df1 utilizando la serie booleana
df_tra[~mask_fil]["ID_MUNICIPIO"].unique()

array(['68020', '15106', '91669', '15131', '68397', '68207', '52427',
       '15401', '68121'], dtype=object)

In [292]:
df_sit_agru[(df_sit_agru["ID_MUNICIPIO"]=="11001")] #[["PROVEEDOR","CODIGO_SITIO"]].groupby("PROVEEDOR").count()

,PROVEEDOR,ID_MUNICIPIO,ANNO,ID_EMPRESA,ID_DEPTO,TRIMESTRE,MUNICIPIO,DEPARTAMENTO,SERVICIO,DESC_TECNOLOGIA_INSTALADA,AZIMUT,ID_TECNOLOGIA_INSTALADA,SECTOR,CODIGO_SITIO
0,Operador 1,11001,2017.0,111,11,4.0,"BOGOTÁ, D.C.",BOGOTÁ D.C.,DATOS,LTE,"40, 55, 145, 315, 205, 155, 230, 4, 335, 170, ...",5,"5, 7, 4, 3, 2, 6, 8, 1, 10, 9","6102, 4916, 5618, 5118, 4728, 4587, 4997, 5128..."
1,Operador 1,11001,2017.0,111,11,4.0,"BOGOTÁ, D.C.",BOGOTÁ D.C.,SMS/VOZ,"HSPA, GSM","302, 40, 55, 145, 315, 39, 205, 155, 230, 76, ...","4, 2","16, 22, 35, 17, 11, 40, 19, 29, 2, 21, 25, 30,...","5179, 5561, 4611, 4728, 5128, 5579, 5061, 4844..."
2,Operador 1,11001,2018.0,111,11,4.0,"BOGOTÁ, D.C.",BOGOTÁ D.C.,DATOS,LTE,"40, 55, 145, 315, 155, 205, 230, 4, 335, 90, 2...",5,"16, 67, 22, 63, 35, 68, 17, 11, 41, 64, 40, 29...","6102, 4916, 5618, 5118, 4611, 11287, 4728, 458..."
3,Operador 1,11001,2018.0,111,11,4.0,"BOGOTÁ, D.C.",BOGOTÁ D.C.,SMS/VOZ,"HSPA, GSM","40, 55, 145, 315, 39, 155, 205, 230, 76, 335, ...","4, 2","16, 67, 63, 64, 40, 2, 55, 38, 39, 48, 53, 57,...","5179, 5561, 4611, 4728, 5128, 5579, 5061, 4844..."
3456,Operador 2,11001,2017.0,112,11,4.0,"BOGOTÁ, D.C.",BOGOTÁ D.C.,DATOS,LTE,"270, 35, 250, 215, 40, 185, 105, 360, 285, 135...",5,"4, 3, 2, 1","7951, 7952, 8096, 7899, 1021, 8087, 7808, 8080..."
3457,Operador 2,11001,2018.0,112,11,4.0,"BOGOTÁ, D.C.",BOGOTÁ D.C.,DATOS,LTE,"270, 35, 250, 215, 255, 40, 185, 105, 285, 135...",5,"4, 3, 2, 1","7951, 7952, 8029, 7899, 7846, 8096, 1021, 8087..."
3591,Operador 3,11001,2017.0,113,11,4.0,"BOGOTÁ, D.C.",BOGOTÁ D.C.,DATOS,LTE,"218, 40, 55, 163, 145, 348, 315, 38, 48, 155, ...",5,"101, 104, 103, 102","6505, 16657, 16673, 17099, 6325, 6349, 16946, ..."
3592,Operador 3,11001,2017.0,113,11,4.0,"BOGOTÁ, D.C.",BOGOTÁ D.C.,SMS/VOZ,"UMTS, GSM","16, 40, 55, 145, 315, 205, 155, 230, 335, 170,...","3, 2","16, 22, 17, 11, 19, 2, 21, 25, 5, 4, 12, 7, 15...","7058, 6678, 6505, 6730, 6337, 6801, 6325, 6369..."
3593,Operador 3,11001,2018.0,113,11,4.0,"BOGOTÁ, D.C.",BOGOTÁ D.C.,DATOS,LTE,"40, 55, 213, 145, 348, 315, 205, 155, 230, 82,...",5,"103, 203, 102, 101, 201, 104, 202","17060, 6505, 16673, 17099, 6325, 6369, 17062, ..."
3594,Operador 3,11001,2018.0,113,11,4.0,"BOGOTÁ, D.C.",BOGOTÁ D.C.,SMS/VOZ,"UMTS, GSM","270, 35, 250, 125, 255, 215, 40, 185, 105, 60,...","3, 2","16, 17, 11, 2, 21, 25, 5, 4, 12, 7, 15, 3, 8, ...","7058, 6678, 6505, 6730, 6337, 6801, 6325, 6369..."


In [293]:
#Procedemos a realizar el cruce de la información
df = pd.merge(df_tra, df_sit_agru, 
         how = "left", 
         left_on = ["PRV", 
                     "ID_MUNICIPIO",
                     "AÑO",
                     "TRIMESTRE", 
                    "SERVICIO_C"], 
         right_on= ["PROVEEDOR", 
                   "ID_MUNICIPIO", 
                   "ANNO", 
                   "TRIMESTRE", 
                    "SERVICIO"])
df

,AÑO,MES,TRIMESTRE,ID_MUNICIPIO,NOM_MUNICIPIO,PRV,PRO,SERVICIO_x,TRÁFICO,SERVICIO_C,...,ID_EMPRESA,ID_DEPTO,MUNICIPIO,DEPARTAMENTO,SERVICIO_y,DESC_TECNOLOGIA_INSTALADA,AZIMUT,ID_TECNOLOGIA_INSTALADA,SECTOR,CODIGO_SITIO
0,2018,10,4,11001,"BOGOTÁ, D.C.",Operador 6,Operador 2,VOZ,1.088575e+08,SMS/VOZ,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2018,11,4,11001,"BOGOTÁ, D.C.",Operador 6,Operador 2,VOZ,1.001573e+08,SMS/VOZ,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2018,9,3,11001,"BOGOTÁ, D.C.",Operador 6,Operador 2,VOZ,9.908555e+07,SMS/VOZ,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2018,8,3,11001,"BOGOTÁ, D.C.",Operador 6,Operador 2,VOZ,9.832873e+07,SMS/VOZ,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2018,5,2,11001,"BOGOTÁ, D.C.",Operador 6,Operador 2,VOZ,9.766652e+07,SMS/VOZ,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
153824,2018,1,1,25875,VILLETA,Operador 1,Operador 6,VOZ,0.000000e+00,SMS/VOZ,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
153825,2018,1,1,19364,JAMBALÓ,Operador 4,Operador 1,VOZ,0.000000e+00,SMS/VOZ,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
153826,2018,2,1,19364,JAMBALÓ,Operador 4,Operador 1,VOZ,0.000000e+00,SMS/VOZ,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
153827,2018,3,1,19364,JAMBALÓ,Operador 4,Operador 1,VOZ,0.000000e+00,SMS/VOZ,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [299]:
df_reducida = df.dropna()
df_reducida

,AÑO,MES,TRIMESTRE,ID_MUNICIPIO,NOM_MUNICIPIO,PRV,PRO,SERVICIO_x,TRÁFICO,SERVICIO_C,...,ID_EMPRESA,ID_DEPTO,MUNICIPIO,DEPARTAMENTO,SERVICIO_y,DESC_TECNOLOGIA_INSTALADA,AZIMUT,ID_TECNOLOGIA_INSTALADA,SECTOR,CODIGO_SITIO
19,2018,10,4,11001,"BOGOTÁ, D.C.",Operador 4,Operador 2,DATOS,6.241267e+07,DATOS,...,114,11,"BOGOTÁ, D.C.",BOGOTÁ D.C.,DATOS,LTE,"40, 55, 145, 348, 315, 155, 205, 230, 57, 335,...",5,"22, 35, 2, 21, 25, 4, 33, 7, 15, 3, 8, 1, 14, ...","8745, 12572, 12678, 12793, 6325, 8681, 2746, 6..."
22,2018,10,4,11001,"BOGOTÁ, D.C.",Operador 4,Operador 2,VOZ,6.230233e+07,SMS/VOZ,...,114,11,"BOGOTÁ, D.C.",BOGOTÁ D.C.,SMS/VOZ,"UMTS, GSM","218, 40, 55, 145, 348, 315, 38, 48, 155, 205, ...","3, 2","22, 11, 2, 21, 25, 27, 5, 4, 12, 7, 15, 20, 3,...","8745, 2584, 12572, 12678, 12793, 8681, 2746, 1..."
24,2018,11,4,11001,"BOGOTÁ, D.C.",Operador 4,Operador 2,VOZ,6.158500e+07,SMS/VOZ,...,114,11,"BOGOTÁ, D.C.",BOGOTÁ D.C.,SMS/VOZ,"UMTS, GSM","218, 40, 55, 145, 348, 315, 38, 48, 155, 205, ...","3, 2","22, 11, 2, 21, 25, 27, 5, 4, 12, 7, 15, 20, 3,...","8745, 2584, 12572, 12678, 12793, 8681, 2746, 1..."
27,2018,12,4,11001,"BOGOTÁ, D.C.",Operador 4,Operador 2,VOZ,6.118135e+07,SMS/VOZ,...,114,11,"BOGOTÁ, D.C.",BOGOTÁ D.C.,SMS/VOZ,"UMTS, GSM","218, 40, 55, 145, 348, 315, 38, 48, 155, 205, ...","3, 2","22, 11, 2, 21, 25, 27, 5, 4, 12, 7, 15, 20, 3,...","8745, 2584, 12572, 12678, 12793, 8681, 2746, 1..."
34,2018,11,4,11001,"BOGOTÁ, D.C.",Operador 4,Operador 2,DATOS,5.679910e+07,DATOS,...,114,11,"BOGOTÁ, D.C.",BOGOTÁ D.C.,DATOS,LTE,"40, 55, 145, 348, 315, 155, 205, 230, 57, 335,...",5,"22, 35, 2, 21, 25, 4, 33, 7, 15, 3, 8, 1, 14, ...","8745, 12572, 12678, 12793, 6325, 8681, 2746, 6..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
153647,2018,11,4,66170,DOSQUEBRADAS,Operador 4,Operador 6,DATOS,1.008034e-03,DATOS,...,114,66,DOSQUEBRADAS,RISARALDA,DATOS,LTE,"270, 215, 40, 180, 220, 230, 80, 100, 320, 170...",5,"5, 4, 3, 2, 6, 1, 13, 14","19650, 19869, 19711, 19670, 19685, 19855, 1987..."
153657,2018,12,4,19845,VILLA RICA,Operador 4,Operador 1,DATOS,8.468630e-04,DATOS,...,114,19,VILLA RICA,CAUCA,DATOS,LTE,"270, 180, 140",5,"4, 5, 6",13972
153677,2018,10,4,8296,GALAPA,Operador 4,Operador 6,DATOS,6.589890e-04,DATOS,...,114,8,GALAPA,ATLÁNTICO,DATOS,LTE,"170, 40, 190, 230, 15, 350, 235, 100",5,"3, 2, 1","2477, 2444, 2533"
153707,2018,10,4,52001,PASTO,Operador 4,Operador 6,DATOS,3.061290e-04,DATOS,...,114,52,PASTO,NARIÑO,DATOS,LTE,"270, 35, 250, 215, 125, 40, 185, 266, 285, 135...",5,"5, 15, 4, 3, 6, 2, 1, 13, 14","14016, 14207, 13773, 14571, 13980, 13812, 1377..."


In [300]:
#Validamos que el tradico sea el correcto en ambas bases
df_tra[["TRÁFICO","NOM_MUNICIPIO"]].groupby("NOM_MUNICIPIO").sum().sum()

TRÁFICO    1.037851e+10
dtype: float64

In [301]:
df[["TRÁFICO","NOM_MUNICIPIO"]].groupby("NOM_MUNICIPIO").sum().sum()

TRÁFICO    1.037851e+10
dtype: float64

In [302]:
df.columns

Index(['AÑO', 'MES', 'TRIMESTRE', 'ID_MUNICIPIO', 'NOM_MUNICIPIO', 'PRV', 'PRO', 'SERVICIO_x', 'TRÁFICO', 'SERVICIO_C', 'PROVEEDOR', 'ANNO', 'ID_EMPRESA', 'ID_DEPTO', 'MUNICIPIO', 'DEPARTAMENTO', 'SERVICIO_y', 'DESC_TECNOLOGIA_INSTALADA', 'AZIMUT', 'ID_TECNOLOGIA_INSTALADA', 'SECTOR', 'CODIGO_SITIO'], dtype='object')

In [303]:
#Eliminamos las columnas repetidas

df = df.drop(columns = ["SERVICIO_C","ANNO","NOM_MUNICIPIO"])
df_reducida = df_reducida.drop(columns = ["SERVICIO_C","ANNO","NOM_MUNICIPIO"])

In [305]:
#Procedemos a crear la base final completa y la versión reducida que será usada para el análisis
df.to_excel("Bases_trafico_x_sitio.xlsx", index=False)
df_reducida.to_excel("Bases_trafico_x_sitio_reducida.xlsx", index=False)

#### Seguimos con el análisis en excel sobre la base la base reducida